## Preprocessing

In [162]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [163]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [164]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [165]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [166]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [application_df['APPLICATION_TYPE'].value_counts().index[x] for x in range(0,application_df['APPLICATION_TYPE'].nunique()) if application_df['APPLICATION_TYPE'].value_counts()[x]<500]

# application_types_to_replace = []
# for i in range(0,application_df['APPLICATION_TYPE'].nunique()):
#     if application_df['APPLICATION_TYPE'].value_counts()[i]<500:
#         application_types_to_replace.append(application_df['APPLICATION_TYPE'].value_counts().index[i])
#     print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [167]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [168]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [application_df['CLASSIFICATION'].value_counts().index[x] for x in range(0,application_df['CLASSIFICATION'].nunique()) if application_df['CLASSIFICATION'].value_counts()[x]<1800]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

## Optimize the Model

In [171]:
# Convert categorical data to numeric with `pd.get_dummies`
charity_dummies = pd.get_dummies(application_df,columns = ["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT",'ASK_AMT','STATUS','SPECIAL_CONSIDERATIONS'])
charity_dummies.head(10)

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ASK_AMT_2310256039,ASK_AMT_3391919220,ASK_AMT_4653011914,ASK_AMT_5591584994,ASK_AMT_8556638692,ASK_AMT_8597806340,STATUS_0,STATUS_1,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
5,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
6,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
7,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
8,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [172]:
# Split our preprocessed data into our features and target arrays
y = charity_dummies["IS_SUCCESSFUL"]
X = charity_dummies.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [173]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [176]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10 # more nodes
hidden_nodes_layer2 = 7 # more nodes


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_63 (Dense)            (None, 10)                87910     
                                                                 
 dense_64 (Dense)            (None, 7)                 77        
                                                                 
 dense_65 (Dense)            (None, 1)                 8         
                                                                 
Total params: 87,995
Trainable params: 87,995
Non-trainable params: 0
_________________________________________________________________


In [177]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [178]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=80) #decreased training epochs

Epoch 1/80
804/804 [==============================] - 3s 3ms/step - loss: 0.6506 - accuracy: 0.6596
Epoch 2/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5044 - accuracy: 0.7750
Epoch 3/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4336 - accuracy: 0.8047
Epoch 4/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4211 - accuracy: 0.8085
Epoch 5/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4167 - accuracy: 0.8087
Epoch 6/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4103 - accuracy: 0.8084
Epoch 7/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4065 - accuracy: 0.8081
Epoch 8/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4050 - accuracy: 0.8083
Epoch 9/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4033 - accuracy: 0.8082
Epoch 10/80
804/804 [==============================] - 2s 3ms/step - loss: 0.4025 - accuracy: 0.8104

In [181]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6222 - accuracy: 0.7214 - 404ms/epoch - 2ms/step
Loss: 0.6221557855606079, Accuracy: 0.7213994264602661


In [180]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")
